Name of Members: 

- __LATOSA, JOSE ROMULO NORIEGA__
- __RAMIREZ, BENMAR SIM GREFALDA__
- __VICTORIA, ALFRED EUGENE TAGLE__

## I. Introduction

League of Legends (LOL), is a MOBA (Multiplayer Online Battle Arena) that's been around since 2009 is one of the most played games globally. 

The dataset we chose for this machine project, is the LOL Worlds 2021 Play-In Group Stats.

For the project we will go through the process of selecting and defining our chosen dataset, performing Exploratory Data Analysis (EDA), data preprocessing and cleaning, model training, HP tuning, model selection and insight extraction.

Our goal is to use the data from the LOL World's 2021 matches to predict the win/loss ratio of the teams and what role is the most important in terms of winning, based on the feature-engineered role performance metrics of the players.

We'll be utilizing machine learning and other techniques discussed in class to build up our predictive model.

## II. Description

The dataset has 220 instances (rows) and 20 features (columns). Each of the features are statistics of either the player or their team, aligned with their specific roles. As well as the gold earned, objectices taken and other important variables.

The data is the game stats for all matches in the League of Legends Worlds 2021 Play-in Groups.

The dataset stated it's collection process was done through lolesports.com, a website which shows all in-depth statistics available for each match, which allowed us to find correlations between in-game statistics and wins.

### List of Variables

| **Variable Name** | **Description**|
|--------------------------------------|----------------|
|**Team** | Acronym Code of Team.|
|**Player** | Nametag of Player.|
|**Opponent** | Acronym code of opposing team in match.|
|**Position**| Position played by player in a match, 5 unique variables.|
|**Champion**| Champion played by a player in a match.|
|**Kills**| Number of kills by player in a match.|
|**Deaths**| Number of deaths by a player in a match.|
|**Assists**| Number of assists by player in match.|
|**Creep Score**| Number of minions and monsters killed by player in a match.|
|**Gold Earned**| Gold earned by player in a match.|
|**Champion Damage** | Percentage of total damage done by team to other champions done by player.|
|**Kill Participation** | Percentage of team kills that player was part of.|
|**Wards Placed** | Number of wards placed by player in match.|
|**Wards Destroyed** | Number of wards killed by player in match.|
|**Wards Interactions** | Sum of wards placed and wards killed by player in match.|
|**Dragons For** | Number of dragons team killed in match.|
|**Dragons Against** | Number of dragons opposing team killed in match.|
|**Barons For** | Number of Barons killed in match.|
|**Barons Against** | Number of Barons opposing team killed in match.|
|**Win or Lose** | Win or Lose (W/L).|

## III. Modules

For this machine project, we will utilize the following Python libraries:

In [2]:
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

## IV. Data Cleaning

We are about to undertake various preprocessing and data cleaning methods to improve the dataset's usability and suitability for our exploratory analysis and model training. 

This process ensures the dataset is ready for use by addressing missing values, duplicates, outliers, and other errors, thereby representing the data accurately.

In the code block below, we will we importing and reading the League of Legends 2021 World Play-in Matches csv file.

In [ ]:
worlds = pd.read_csv("LOL2021WORLDS.csv")
worlds.head()

**Checking for Missing Values**

## Exploratory Data Analysis

## Model Training

## Improving Model

## Model Selection

## Insights and Conclusions

## References